In [4]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, precision_recall_fscore_support
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from catboost.utils import eval_metric

SOTAB CTA

In [447]:
sotab_cta_train = pd.read_csv('SOTAB CTA/sotab_v2_cta_training_set.csv')
sotab_cta_test = pd.read_csv('SOTAB CTA/sample_test.csv')

In [ ]:
sotab_cta_test

In [449]:
final_test_table=pd.DataFrame()
table=sotab_cta_test
primary_path='SOTAB CTA/Test/'


all_data_from_all_columns = []
all_target = []
for table_index, table_values in table.iterrows():

    full_path = primary_path + table_values[0]
    index_of_desired_columns = table_values[1]
    label_of_table = table_values[2]

    df = pd.read_json(full_path, compression='gzip', lines=True) 

    all_values_from_desired_columns = []

    for value in df[index_of_desired_columns]:
        if value != None:
            all_values_from_desired_columns.append(value)

    joined_values = ', '.join(map(str, all_values_from_desired_columns))

    all_data_from_all_columns.append(joined_values)

    all_target.append(label_of_table)

final_test_table['data']=all_data_from_all_columns
final_test_table['target']=all_target


In [ ]:
final_test_table

In [451]:
final_train_table=pd.DataFrame()
table=sotab_cta_train
primary_path='SOTAB CTA/Train/'


all_data_from_all_columns = []
all_target = []
for table_index,table_values in table.iterrows():

    full_path = primary_path + table_values[0]
    index_of_desired_columns = table_values[1]
    label_of_table = table_values[2]


    df = pd.read_json(full_path, compression='gzip', lines=True) 
    all_values_from_desired_columns = []

    for value in df[index_of_desired_columns]:
        if value != None:
            all_values_from_desired_columns.append(value)

    joined_values = ', '.join(map(str, all_values_from_desired_columns))

    all_data_from_all_columns.append(joined_values)        
    all_target.append(label_of_table)

final_train_table['data']=all_data_from_all_columns
final_train_table['target']=all_target

In [457]:
final_train_table

,data,target,encoded_label
0,"2020-07-10, 2016-04-08, 2013-09-13, 2016-08-05...",Date,0
1,"The Sleep Revolution, Viva, Ame, Lidere, The S...",Book/name,1
2,"en-US, pt-BR, pt-BR, pt-BR, pt-BR, pt-BR, pt-B...",Language,2
3,"Homer William Bedell Stanford W. Stanford, Dav...",Person/name,3
4,"Paperback, Hardcover, Paperback, None, Paperba...",BookFormatType,4
...,...,...,...
116882,"Billions, Bonanza, Voltron: Legendary Defender...",CreativeWorkSeries,81
116883,"Family Guy Season 2 Episode 3, Naked and Afrai...",TVEpisode/name,80
116884,"Family Guy, Naked and Afraid XL, Animal Kingdo...",CreativeWorkSeries,81
116885,"Psych Season 2 Episode 8, Still a Mystery Seas...",TVEpisode/name,80


In [458]:
types = list(final_train_table["target"].unique())
 
final_train_table['encoded_label'] = final_train_table['target'].apply(lambda row: types.index(row))
final_test_table['encoded_label'] = final_test_table['target'].apply(lambda row: types.index(row))

In [459]:
len(final_test_table.encoded_label.unique())

82

In [460]:
final_train_table

,data,target,encoded_label
0,"2020-07-10, 2016-04-08, 2013-09-13, 2016-08-05...",Date,0
1,"The Sleep Revolution, Viva, Ame, Lidere, The S...",Book/name,1
2,"en-US, pt-BR, pt-BR, pt-BR, pt-BR, pt-BR, pt-B...",Language,2
3,"Homer William Bedell Stanford W. Stanford, Dav...",Person/name,3
4,"Paperback, Hardcover, Paperback, None, Paperba...",BookFormatType,4
...,...,...,...
116882,"Billions, Bonanza, Voltron: Legendary Defender...",CreativeWorkSeries,81
116883,"Family Guy Season 2 Episode 3, Naked and Afrai...",TVEpisode/name,80
116884,"Family Guy, Naked and Afraid XL, Animal Kingdo...",CreativeWorkSeries,81
116885,"Psych Season 2 Episode 8, Still a Mystery Seas...",TVEpisode/name,80


In [461]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=3)
X_train = tfidf.fit_transform(final_train_table['data'])
X_test = tfidf.transform(final_test_table['data'])

X_test.shape

(318, 344692)

In [ ]:
X_train.shape

In [ ]:
values = list(final_train_table['encoded_label'])
n_values = np.max(values) + 1
y_train = np.eye(n_values)[values]
y_train.shape

In [ ]:
y_train

In [498]:
values = list(final_test_table['encoded_label'])
y_test = np.eye(n_values)[values]
y_test.shape

(318, 82)

In [500]:
from sklearn.feature_selection import SelectKBest, chi2

chi_square = SelectKBest(chi2, k=1000)
X_train = chi_square.fit_transform(X_train, y_train)
X_test = chi_square.transform(X_test)

In [ ]:
feature_names = chi_square.get_feature_names_out()
print("Number of features:", len(feature_names))
print("Some example features:", feature_names[:10])

In [ ]:
model = CatBoostClassifier(
    loss_function='MultiLogloss',
    learning_rate=0.2,
    iterations=100
)
model.fit(X_train, y_train)

In [509]:
y_pred = model.predict(X_test)

In [ ]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [511]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test, y_pred, average="macro")

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [488]:
micro_f1 = f1_score(y_test, y_pred, average="micro")

In [142]:
import pickle
with open('CatBoost Evaluation model/sotab-CTA-model2.pkl', 'wb') as file:
    pickle.dump(model, file)

SOTAB CPA

In [143]:
sotab_cpa_train = pd.read_csv('SOTAB CPA/sotab_v2_cpa_training_set.csv')
sotab_cpa_test = pd.read_csv('SOTAB CPA/sample_test.csv')

In [144]:
sotab_cpa_test

,table_name,main_column_index,column_index,label
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone
...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions


In [147]:
import re
gt = {'train':{}, 'test':{}}
for index, row in sotab_cpa_train.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in sotab_cpa_test.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]
    
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    

    text = re.sub(' +', ' ', str(text)).strip()
    
    return text


def get_all_table_columns(file_name, index):
    

    if file_name in sotab_cpa_train["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    

    cleaned_main="{}{}"

    x = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) 
    cleaned_main=cleaned_main.format("", x)
    cleaned_rows.append(cleaned_main)

    cleaned_rows.append("")
    
    
    for row in df.iloc[:, index].tolist():
        get_values_var=7
        cleaned = " ".join(clean_text(row).split()[:20])
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
        else:
            get_values_var=get_values_var-1

    
    return " ".join(cleaned_rows[:get_values_var])

In [148]:
test_examples = []
for table in gt['test']:
    for column in gt['test'][table]:
        col_str = get_all_table_columns(table, column)
        test_examples.append([table, column, col_str, gt['test'][table][column], table.split("_")[0]])

In [ ]:
test_examples

In [150]:
final_table_sotab_cpa=pd.DataFrame()
labels=[]
values=[]
for i in test_examples:
    labels.append(i[-2])
    values.append(i[2])

final_table_sotab_cpa['data']=values
final_table_sotab_cpa['cpa label']=labels

In [151]:
test_table_sotab_cpa=final_table_sotab_cpa

test_table_sotab_cpa=test_table_sotab_cpa

In [152]:
train_examples = []
for table in gt['train']:
    for column in gt['train'][table]:
        col_str = get_all_table_columns(table, column)
        train_examples.append([table, column, col_str, gt['train'][table][column], table.split("_")[0]])

In [ ]:
train_examples

In [293]:
final_table_sotab_cpa=pd.DataFrame()
labels=[]
values=[]
for i in train_examples:
    labels.append(i[-2])
    values.append(i[2])

final_table_sotab_cpa['data']=values
final_table_sotab_cpa['cpa label']=labels

In [294]:
train_table_sotab_cpa=final_table_sotab_cpa

train_table_sotab_cpa=train_table_sotab_cpa

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=3)
X_train_cpa_sotab = tfidf.fit_transform(train_table_sotab_cpa['data']) 
X_test_cpa_sotab = tfidf.transform(test_table_sotab_cpa['data'])

X_test_cpa_sotab.shape

In [332]:
types = list(train_table_sotab_cpa["cpa label"].unique())
 
train_table_sotab_cpa['encoded_label'] = train_table_sotab_cpa['cpa label'].apply(lambda row: types.index(row))
test_table_sotab_cpa['encoded_label'] = test_table_sotab_cpa['cpa label'].apply(lambda row: types.index(row))

In [ ]:
values = list(train_table_sotab_cpa['encoded_label'])
n_values = np.max(values) + 1
y_train_cpa_sotab = np.eye(n_values)[values]
y_train_cpa_sotab.shape

In [ ]:
values = list(test_table_sotab_cpa['encoded_label'])
n_values = np.max(values) + 1
y_test_cpa_sotab = np.eye(n_values)[values]
y_test_cpa_sotab.shape

In [339]:
from sklearn.feature_selection import SelectKBest, chi2

chi_square = SelectKBest(chi2, k=5000)
X_train_cpa_sotab = chi_square.fit_transform(X_train_cpa_sotab, y_train_cpa_sotab)
X_test_cpa_sotab = chi_square.transform(X_test_cpa_sotab)

In [ ]:
feature_names = tfidf.get_feature_names_out()
print("Number of features:", len(feature_names))
print("Some example features:", feature_names[:10])

In [ ]:
feature_names = chi_square.get_feature_names_out()
print("Number of features:", len(feature_names))
print("Some example features:", feature_names[:10])

In [342]:
X_test_cpa_sotab.shape

(509, 5000)

In [343]:
X_train_cpa_sotab.shape

(109994, 5000)

In [ ]:
model_cpa_sotab = CatBoostClassifier(
    loss_function='MultiLogloss',
    learning_rate=0.2,
    iterations=100
)
model_cpa_sotab.fit(X_train_cpa_sotab, y_train_cpa_sotab)

In [417]:
y_pred_cpa_sotab = model_cpa_sotab.predict(X_test_cpa_sotab)

In [418]:
print("Classification Report:")
print(classification_report(y_test_cpa_sotab, y_pred_cpa_sotab))

Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.22      0.27         9
           1       1.00      0.11      0.20         9
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         6
           4       0.75      1.00      0.86         6
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00        12
           7       1.00      0.50      0.67         4
           8       0.75      0.60      0.67         5
           9       0.00      0.00      0.00         7
          10       0.58      0.58      0.58        12
          11       1.00      0.25      0.40         8
          12       0.00      0.00      0.00         1
          13       1.00      0.43      0.60         7
          14       1.00      0.62      0.77         8
          15       1.00      0.88      0.93         8
          16       1.00      0.17      0.29         6
    

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [411]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test_cpa_sotab, y_pred_cpa_sotab, average="macro")

c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [413]:
micro_f1 = f1_score(y_test_cpa_sotab, y_pred_cpa_sotab, average="micro")

In [357]:
import pickle
with open('CatBoost Evaluation model/sotab-CPA-model.pkl', 'wb') as file:
    pickle.dump(model_cpa_sotab, file)

WikiTable CTA MULTI-LABEL CLASSIFIER

In [185]:
from catboost import CatBoostClassifier
from catboost.utils import eval_metric

In [186]:
cta_train = pd.read_json('WikiData/train.table_col_type.json')
cta_test = pd.read_json('WikiData/test.table_col_type.json')

In [188]:
cta_all_labels = ['soccer.football_league', 'government.government_office_or_title', 'organization.non_profit_organization', 'olympics.olympic_games', 'cvg.cvg_genre', 'ice_hockey.hockey_position', 'tv.tv_network', 'aviation.airline', 'american_football.football_conference', 'soccer.football_world_cup', 'american_football.football_coach', 'military.military_unit', 'military.military_post', 'music.media_format', 'tv.tv_personality', 'baseball.baseball_team', 'cvg.cvg_developer', 'soccer.football_award', 'ice_hockey.hockey_team', 'tv.tv_writer', 'meteorology.tropical_cyclone_season', 'soccer.fifa', 'cvg.cvg_publisher', 'baseball.baseball_player', 'sports.sports_championship', 'soccer.football_team_manager', 'sports.golfer', 'baseball.baseball_position', 'military.rank', 'cvg.cvg_platform', 'music.musical_group', 'amusement_parks.ride', 'music.genre', 'music.lyricist', 'music.record_label', 'meteorology.tropical_cyclone', 'aviation.airport']

In [ ]:
final_test_table=pd.DataFrame()
all_values=[]
labelsx=[]
for index_cta, row_cta in cta_test.iterrows():
    
    for table_index,table_row in enumerate(row_cta[6]):
        labels=[]
        for label in  row_cta[7][table_index]:
            if label in cta_all_labels:
                labels.append(label)
        

        if len(labels)>0:
            value_list=[]
            for value_index, value in table_row:
                value_list.append(value[1])
            joined_value_list = ', '.join(map(str, value_list))   
            
            all_values.append(joined_value_list)

            joined_label=labels
            labelsx.append(joined_label)

final_test_table['data']=all_values
final_test_table['Target']=labelsx

print(len(all_values))
print(len(labelsx))

In [ ]:
final_test_table

In [ ]:
final_train_table=pd.DataFrame()
all_values=[]
labelsx=[]
for index_cta, row_cta in cta_train.iterrows():
    
    for table_index,table_row in enumerate(row_cta[6]):
        labels=[]
        for label in  row_cta[7][table_index]:
            if label in cta_all_labels:
                labels.append(label)

        if len(labels)>0:
            value_list=[]
            for value_index, value in table_row:
                value_list.append(value[1])
            joined_value_list = ', '.join(map(str, value_list))   
            
            all_values.append(joined_value_list)
            joined_label=labels
            labelsx.append(joined_label)

final_train_table['data']=all_values
final_train_table['Target']=labelsx

print(len(all_values))
print(len(labelsx))

In [ ]:
final_train_table

In [196]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=3)
X_train = tfidf.fit_transform(final_train_table['data'])
X_test = tfidf.transform(final_test_table['data'])

X_test.shape


(807, 19252)

In [197]:
types =[]
for value in final_train_table['Target']:
    for i in value:
        types.append(i)
types=list(set(types))

indexes=[]
for value in final_train_table['Target']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

final_train_table['encoded_label']=indexes


In [ ]:
final_train_table

In [199]:
valuesx = list(final_train_table['encoded_label'])
find_max=[]
for x in valuesx:
    for i in x:
        find_max.append(i)
n_values = np.max(find_max) + 1
target_encoded=[]
for values in valuesx:
    y_train = [0] * n_values
    for i in values:
        y_train =y_train+ np.eye(n_values)[i]
    target_encoded.append(y_train)

In [202]:
y_train = target_encoded

In [203]:
final_train_table['encoded_matrix']=y_train

In [ ]:
final_train_table

In [205]:
indexes=[]
for value in final_test_table['Target']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)
final_test_table['encoded_label']=indexes

In [ ]:
final_test_table

In [207]:
valuesx = list(final_test_table['encoded_label'])
target_encoded=[]
for values in valuesx:
    y_test = [0] * n_values
    for i in values:
        y_test =y_test+ np.eye(n_values)[i]
    target_encoded.append(y_test)


In [208]:
y_test = target_encoded

In [210]:
final_test_table['encoded_matrix']=y_test

In [211]:
final_test_table

,data,Target,encoded_label,encoded_matrix
0,"Premier League, Premier League, League One, Ch...",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"Uni-President Lions, Wei Chuan Dragons, Uni-Pr...",[baseball.baseball_team],[21],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"Wei Chuan Dragons, Sinon Bulls, Sinon Bulls, U...",[baseball.baseball_team],[21],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Football, Football, Football, Ice hockey, Ice ...",[cvg.cvg_genre],[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
4,"Joel Skinner, Rick Rodriguez, Gerald Perry",[baseball.baseball_player],[0],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
802,"J. League Division 2, J. League Division 2, J....",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
803,"J. League Division 1, J. League Division 1, J....",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
804,"J. League Division 1, J. League Division 2, J....",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
805,"Las Vegas 51s, Jacksonville Suns, Vero Beach D...",[baseball.baseball_team],[21],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [212]:
final_train_table

,data,Target,encoded_label,encoded_matrix
0,"Persian Gulf Cup, Persian Gulf Cup, Azadegan L...",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"Pro League, Pro League, Pro League, Pro League...",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"Ron Villone, Bob Wolcott, Chris Widger, Andy S...",[baseball.baseball_player],[0],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"Ligat ha'Al, Ligat ha'Al, Ligat ha'Al, Liga Le...",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Nine Network, ARD, RTL plus, RCTI, RCTI, RCTI,...",[tv.tv_network],[30],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
38715,"Peter Gwinn and Rob Dubbin, Peter Gwinn and La...",[tv.tv_writer],[5],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
38716,"Argentine Primera, Argentine Primera, Argentin...",[soccer.football_league],[23],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38717,"Seven Network, Dubai TV, ZDF, TVN, NBC",[tv.tv_network],[30],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
38718,"737 NAS, 800 NAS, 801 NAS, 809 NAS, 815 NAS, 8...",[military.military_unit],[10],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
clf = CatBoostClassifier(
    loss_function='MultiLogloss',
    #eval_metric='HammingLoss',
    iterations=100,
    learning_rate=0.2
)
clf.fit(X_train, y_train)

In [215]:
y_pred = clf.predict(X_test)

In [ ]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test, y_pred, average="macro")

In [225]:
micro_f1 = f1_score(y_test, y_pred, average="micro")

In [231]:
micro_f1

0.7370753323485968

In [232]:
macro

0.5476315873897606

In [233]:
precision

0.7117339543354456

In [236]:
recall

0.461617358145046

In [230]:
with open('CatBoost Evaluation model/wikiTables-CTA-model.pkl', 'wb') as file:
    pickle.dump(clf, file)

WikiTable CPA

In [90]:
cpa_train = pd.read_json('WikiData/train.table_rel_extraction.json')
cpa_test = pd.read_json('WikiData/test.table_rel_extraction.json')

In [91]:
cpa_all_labels = ['time.event.locations', 'music.artist.album', 'sports.sports_team.sport', 'baseball.baseball_team.league', 'tv.tv_program.country_of_origin', 'music.album.artist', 'sports.sports_team.location', 'time.event.instance_of_recurring_event', 'aviation.airline.hubs', 'sports.sports_championship_event.champion', 'sports.sports_facility.teams', 'baseball.baseball_player.position_s', 'sports.sports_league.teams-sports.sports_league_participation.team', 'tv.tv_network.programs-tv.tv_network_duration.program', 'sports.sports_league_season.league', 'olympics.olympic_athlete.country-olympics.olympic_athlete_affiliation.country', 'american_football.football_player.position_s', 'music.composer.compositions', 'meteorology.tropical_cyclone.tropical_cyclone_season', 'cvg.computer_videogame.developer', 'tv.tv_character.appeared_in_tv_program-tv.regular_tv_appearance.actor', 'cvg.computer_videogame.publisher', 'soccer.football_player.position_s', 'tv.tv_program.original_network-tv.tv_network_duration.network', 'music.composition.composer', 'ice_hockey.hockey_player.hockey_position', 'book.author.works_written', 'film.film.genre', 'film.film.directed_by', 'film.film.produced_by', 'film.film.language', 'broadcast.broadcast.area_served', 'award.award_category.category_of', 'location.location.nearby_airports', 'location.country.official_language']

In [ ]:
cpa_test

In [93]:
test_examples = []

for primary_row_index, primary_row_values in cpa_test.iterrows():
    new_col_labels = []
    for col_index, col_labels in enumerate(primary_row_values[7]):
        sampled_labels = []
        for label in col_labels:
            if label in cpa_all_labels:
                sampled_labels.append(label)
        
        if len(sampled_labels) !=0:
            new_col_labels.append(sampled_labels)

            column_string = ""

            for row_index in primary_row_values[6][0][:5]:
                column_string += f"{row_index[1][1]} "
            column_string = column_string.strip()

            column_string += ""
        
            for row_index in primary_row_values[6][col_index+1][:5]:
                column_string += f"{row_index[1][1]} "
            column_string = column_string.strip()

            test_examples.append([primary_row_values[0], col_index, column_string, sampled_labels])

In [ ]:
test_examples

In [ ]:
train_examples = []

for primary_row_index, primary_row_values in cpa_train.iterrows():
    new_col_labels = []
    for col_index, col_labels in enumerate(primary_row_values[7]):
        sampled_labels = []
        for label in col_labels:
            if label in cpa_all_labels:
                sampled_labels.append(label)
        
        if len(sampled_labels) !=0:
            new_col_labels.append(sampled_labels)

            column_string = ""

            for row_index in primary_row_values[6][0][:5]:
                column_string += f"{row_index[1][1]} "
            column_string = column_string.strip()

            column_string += ""
        
            for row_index in primary_row_values[6][col_index+1][:5]:
                column_string += f"{row_index[1][1]} "
            column_string = column_string.strip()

            train_examples.append([primary_row_values[0], col_index, column_string, sampled_labels])

In [95]:
final_table_cpa=pd.DataFrame()
labels=[]
values=[]
for i in test_examples:
    labels.append(i[-1])
    values.append(i[2])

final_table_cpa['data']=values
final_table_cpa['cpa label']=labels

In [ ]:
final_table_cpa=pd.DataFrame()
labels=[]
values=[]
for i in train_examples:
    labels.append(i[-1])
    values.append(i[2])

final_table_cpa['data']=values
final_table_cpa['cpa label']=labels

In [96]:
train_table_cpa=final_table_cpa
train_table_cpa=train_table_cpa

In [97]:
test_table_cpa=final_table_cpa
test_table_cpa=test_table_cpa

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=3)
X_train_cpa = tfidf.fit_transform(train_table_cpa['data'])
X_test_cpa = tfidf.transform(test_table_cpa['data'])

X_test_cpa.shape

In [ ]:
X_train_cpa.shape

In [100]:
types =[]
for value in train_table_cpa['cpa label']:
    for i in value:
        types.append(i)
types=list(set(types))

indexes=[]
for value in train_table_cpa['cpa label']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

train_table_cpa['encoded_label']=indexes

In [102]:
valuesx = list(train_table_cpa['encoded_label'])

find_max=[]
for x in valuesx:
    for i in x:
        find_max.append(i)

n_values = np.max(find_max) + 1

target_encoded=[]
for values in valuesx:
    y_train = [0] * n_values
    for i in values:

        y_train =y_train+ np.eye(n_values)[i]
    target_encoded.append(y_train)


In [103]:
y_train_cpa=target_encoded

In [105]:
indexes=[]
for value in test_table_cpa['cpa label']:
    cell=[]
    for i in value:
        cell.append(types.index(i))
    indexes.append(cell)

test_table_cpa['encoded_label']=indexes

valuesx = list(test_table_cpa['encoded_label'])

target_encoded=[]
for values in valuesx:
    y_test = [0] * n_values
    for i in values:

        y_test =y_test+ np.eye(n_values)[i]
    target_encoded.append(y_test)

In [151]:
y_test_cpa=target_encoded

In [ ]:
clf_cpa = CatBoostClassifier(
    loss_function='MultiLogloss',
    #eval_metric='HammingLoss',
    iterations=100,
    learning_rate=0.2
)
clf_cpa.fit(X_train_cpa, y_train_cpa)

In [167]:
y_pred_cpa = clf_cpa.predict(X_test_cpa)

In [169]:
accuracy = eval_metric(y_test_cpa, y_pred_cpa, 'Accuracy')[0]
print(f'Accuracy: {accuracy}')

Accuracy: 0.7214953271028037


In [ ]:
accuracy_per_class = eval_metric(y_test_cpa, y_pred_cpa, 'Accuracy:type=PerClass')
for cls, value in zip(clf.classes_, accuracy_per_class):
    print(f'Accuracy for class {cls}: {value}')

In [ ]:
for metric in ('Precision', 'Recall', 'F1'):
    print(metric)
    values = eval_metric(y_test_cpa, y_pred_cpa, metric)
    for cls, value in zip(clf.classes_, values):
        print(f'class={cls}: {value:.4f}')
    print()

In [ ]:
print("Classification Report:")
print(classification_report(y_test_cpa, y_pred_cpa))

In [ ]:
precision, recall, macro, _ = precision_recall_fscore_support(y_test_cpa, y_pred_cpa, average="macro")

In [175]:
micro_f1 = f1_score(y_test_cpa, y_pred_cpa, average="micro")

In [181]:
macro

0.5917938311691

In [182]:
micro_f1

0.8385026737967914

In [183]:
recall

0.5024201991250914

In [184]:
precision

0.7922813569872393

In [180]:
import pickle
with open('CatBoost Evaluation model/wikiTables-CPA-model.pkl', 'wb') as file:
    pickle.dump(clf_cpa, file)